In [18]:
import skrub
import skrub.datasets
from monkey_patching_v02_data_provenance import set_provenance, enter_provenance_mode_dataop, enter_provenance_mode_var
set_provenance(skrub._data_ops._evaluation,"evaluate", provenance_func=enter_provenance_mode_dataop)
set_provenance(skrub._data_ops._data_ops.Var,"compute", provenance_func=enter_provenance_mode_var)
dataset = skrub.datasets.fetch_credit_fraud()
skrub.TableReport(dataset.baskets)

Processing column   2 / 2


<TableReport: use .open() to display>

In [19]:
skrub.TableReport(dataset.products)

Processing column   7 / 7


,,,,,,,


In [20]:
products = skrub.var("products", dataset.products)
baskets = skrub.var("baskets", dataset.baskets)

basket_ids = baskets[["ID"]].skb.mark_as_X()
fraud_flags = baskets["fraud_flag"].skb.mark_as_y()

[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
          ID  fraud_flag  _prov
1      51113           0      0
7      41798           0      1
9      39361           0      2
13     38615           0      3
14     70262           0      4
...      ...         ...    ...
92785  21243           0  61236
92786  45891           0  61237
92787  42613           0  61238
92788  43567           0  61239
92789  68268           0  61240

[61241 rows x 3 columns]
[PROVENANCE VAR]: End
          ID  fraud_flag  _prov
1      51113           0      0
7      41798           0      1
9      39361           0      2
13     38615           0      3
14     70262           0      4
...      ...         ...    ...
92785  21243           0  61236
92786  45891           0  61237
92787  42613           0  61238


In [21]:
kept_products = products[products["basket_ID"].isin(basket_ids["ID"])]
products_with_total = kept_products.assign(
    total_price=kept_products["Nbr_of_prod_purchas"] * kept_products["cash_price"]
)
products_with_total

[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567               

<CallMethod 'assign'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA         799           LG   

                                                    model goods_code  \
1       APPLE WATCH SERIES 6 GPS 44MM SPACE GREY ALUMI...  239001518   
9       2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...  239246780   
11      2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...  239246776   
15      APPLE WATCH SERIES 6 GPS 40MM BLUE ALUMINIUM CASE  239001540   
16      2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16G...  240575990   
...                                                   ...        ...   
163352  SILENTNIGHT SLEEP GENIUS FULL HEIGHT HEADBOARD...  236938439   
163353  LG OUTDOOR BERGEN 2-SEAT GARDEN SIDE TABLE REC...  239742814   
163354  2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-F...  240040978   
163355  2020 APPLE IMAC 27 ALL-IN-ONE INTEL CORE I7 8G...  238923518   
163356  LG OLED48A16LA 2021 OLED HDR 4K ULTRA HD SMART TV  239866717   

        Nbr_of_prod_purchas            _prov  total_price  
1                         1  281474976710656          409  
9                         1  281474976710657         1187  
11                        1  281474976710658          898  
15                        1  281474976710659          379  
16                        1  281474976710660         1899  
...                     ...              ...          ...  
163352                    1  281474976820031          259  
163353                    1  281474976820032          949  
163354                    1  281474976820033         1099  
163355                    1  281474976820034         2099  
163356                    1  281474976820035          799  

[109380 rows x 9 columns]

In [22]:
n = skrub.choose_int(5, 15, name="n_components")
encoder = skrub.choose_from(
    {
        "MinHash": skrub.MinHashEncoder(n_components=n),
        "LSA": skrub.StringEncoder(n_components=n),
    },
    name="encoder",
)
vectorizer = skrub.TableVectorizer(high_cardinality=encoder)

In [23]:
vectorized_products = products_with_total.skb.apply(
    vectorizer, exclude_cols="basket_ID"
)

[PROVENANCE]: Start
    X = <CallMethod 'assign'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA   

In [24]:
aggregated_products = vectorized_products.groupby("basket_ID").agg("mean").reset_index()

augmented_baskets = basket_ids.merge(
    aggregated_products, left_on="ID", right_on="basket_ID"
).drop(columns=["ID", "basket_ID"])
aggregated_products

[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END


c:\Users\teodo\Desktop\github\rdepro_skrub_provenance\monkey_patching_v02\data_provenance\monkey_patching_v02_data_provenance.py:192: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  groupby_keys = object_inside_preview.grouper.names


[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END


<CallMethod 'reset_index'>
Result:
―――――――
       basket_ID        item_0        item_1        item_2        item_3  \
0              0 -1.920020e+09 -1.925957e+09 -1.950732e+09 -2.089279e+09   
1              1 -1.965640e+09 -2.068358e+09 -1.951644e+09 -2.061611e+09   
2              2 -1.944861e+09 -2.140802e+09 -1.811393e+09 -2.061611e+09   
3              4 -2.028069e+09 -2.094152e+09 -2.117105e+09 -2.120218e+09   
4              5 -2.147303e+09 -2.140802e+09 -2.119336e+09 -2.046307e+09   
...          ...           ...           ...           ...           ...   
61236      76538 -2.035695e+09 -2.009446e+09 -2.069869e+09 -2.057859e+09   
61237      76539 -1.876016e+09 -2.079762e+09 -2.000360e+09 -2.067944e+09   
61238      76540 -2.119082e+09 -2.092438e+09 -2.091895e+09 -2.096070e+09   
61239      76541 -1.797199e+09 -1.627835e+09 -2.062356e+09 -1.882234e+09   
61240      76543 -2.028069e+09 -1.842717e+09 -1.969389e+09 -2.095685e+09   

             item_4        item_5        item_6        item_7        item_8  \
0     -2.013007e+09 -1.771374e+09 -1.383638e+09 -1.961485e+09 -1.962552e+09   
1     -1.962231e+09 -1.997761e+09 -2.038879e+09 -2.009485e+09 -2.032688e+09   
2     -1.956563e+09 -2.115316e+09 -2.002458e+09 -2.009485e+09 -1.993392e+09   
3     -2.038419e+09 -2.091591e+09 -2.057701e+09 -1.971896e+09 -2.081421e+09   
4     -1.954553e+09 -2.115316e+09 -2.048164e+09 -1.930491e+09 -1.950004e+09   
...             ...           ...           ...           ...           ...   
61236 -1.951792e+09 -2.049706e+09 -1.882471e+09 -1.959163e+09 -2.034154e+09   
61237 -1.974285e+09 -1.583488e+09 -1.417763e+09 -1.970080e+09 -2.053228e+09   
61238 -2.053898e+09 -2.069353e+09 -2.075300e+09 -2.009485e+09 -2.071984e+09   
61239 -1.890213e+09 -1.917350e+09 -1.803504e+09 -1.702801e+09 -2.082846e+09   
61240 -2.042481e+09 -2.073396e+09 -1.700986e+09 -1.942188e+09 -1.940084e+09   

       ...  goods_code_3  goods_code_4  goods_code_5  goods_code_6  \
0      ... -1.949358e+09 -1.827033e+09 -2.055218e+09 -1.816320e+09   
1      ... -1.841996e+09 -2.009235e+09 -1.681195e+09 -1.800083e+09   
2      ... -2.002524e+09 -2.128519e+09 -2.039266e+09 -1.930114e+09   
3      ... -2.110325e+09 -1.997428e+09 -2.018729e+09 -1.980666e+09   
4      ... -1.965455e+09 -2.066065e+09 -1.808221e+09 -1.964930e+09   
...    ...           ...           ...           ...           ...   
61236  ... -1.899165e+09 -1.682875e+09 -1.847190e+09 -1.824878e+09   
61237  ... -1.927355e+09 -1.565454e+09 -1.926871e+09 -1.697931e+09   
61238  ... -1.953058e+09 -2.081421e+09 -1.800211e+09 -1.380544e+09   
61239  ... -1.842336e+09 -1.937188e+09 -1.797751e+09 -1.853597e+09   
61240  ... -1.797887e+09 -2.056242e+09 -2.062854e+09 -1.949028e+09   

       goods_code_7  goods_code_8  goods_code_9  Nbr_of_prod_purchas  \
0     -1.824060e+09 -2.044037e+09 -1.808573e+09                  1.0   
1     -1.931498e+09 -1.799015e+09 -1.917557e+09                  1.0   
2     -2.086585e+09 -2.040257e+09 -1.922351e+09                  1.0   
3     -1.844453e+09 -1.947941e+09 -1.716814e+09                  1.0   
4     -2.067649e+09 -1.428901e+09 -2.023444e+09                  1.0   
...             ...           ...           ...                  ...   
61236 -1.736842e+09 -1.959855e+09 -1.931548e+09                  1.0   
61237 -1.929229e+09 -2.086669e+09 -2.023647e+09                  1.0   
61238 -1.558873e+09 -1.503619e+09 -2.007342e+09                  1.0   
61239 -1.658452e+09 -1.917024e+09 -1.939035e+09                  1.0   
61240 -1.852831e+09 -1.997816e+09 -1.753011e+09                  1.0   

       total_price                                              _prov  
0       431.666656  [281474976753540, 281474976753541, 28147497675...  
1       524.000000                 [281474976736084, 281474976736085]  
2       994.500000                 [281474976798105, 281474976798106]  
3       499.500000                 [281474976757353, 281474976757354]  
4       749.000000           

Finally, we add a supervised estimator, and use |choose_float| to
add the learning rate as a hyperparameter to tune.



In [25]:
augmented_baskets.columns

[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END
[PROVENANCE]: Start
[PROVENANCE]: END


<GetAttr 'columns'>
Result:
―――――――
Index(['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6',
       'item_7', 'item_8', 'item_9', 'cash_price', 'make_0', 'make_1',
       'make_2', 'make_3', 'make_4', 'make_5', 'make_6', 'make_7', 'make_8',
       'make_9', 'model_0', 'model_1', 'model_2', 'model_3', 'model_4',
       'model_5', 'model_6', 'model_7', 'model_8', 'model_9', 'goods_code_0',
       'goods_code_1', 'goods_code_2', 'goods_code_3', 'goods_code_4',
       'goods_code_5', 'goods_code_6', 'goods_code_7', 'goods_code_8',
       'goods_code_9', 'Nbr_of_prod_purchas', 'total_price', '_prov'],
      dtype='object')

In [26]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(
    learning_rate=skrub.choose_float(0.01, 0.9, log=True, name="learning_rate")
)

predictions = augmented_baskets.skb.apply(hgb, y=fraud_flags)
predictions

[PROVENANCE]: Start
    X = <CallMethod 'drop'>
Result:
―――――――
             item_0        item_1        item_2        item_3        item_4  \
0     -2.119082e+09 -2.092438e+09 -2.091895e+09 -2.096070e+09 -2.053898e+09   
1     -2.119082e+09 -1.621485e+09 -1.795711e+09 -2.069713e+09 -2.053898e+09   
2     -2.119082e+09 -1.621485e+09 -1.795711e+09 -2.069713e+09 -2.053898e+09   
3     -2.119082e+09 -2.092438e+09 -2.091895e+09 -2.096070e+09 -2.053898e+09   
4     -2.119082e+09 -1.621485e+09 -1.795711e+09 -2.069713e+09 -2.053898e+09   
...             ...           ...           ...           ...           ...   
61236 -2.028069e+09 -2.078193e+09 -2.117481e+09 -2.108864e+09 -2.042481e+09   
61237 -2.119082e+09 -1.621485e+09 -1.795711e+09 -2.069713e+09 -2.053898e+09   
61238 -1.944861e+09 -2.140802e+09 -1.965139e+09 -1.988736e+09 -1.956563e+09   
61239 -2.119082e+09 -1.621485e+09 -1.795711e+09 -2.069713e+09 -2.053898e+09   
61240 -2.128260e+09 -2.095576e+09 -2.144446e+09 -2.020459e+09 -2.05

<Apply HistGradientBoostingClassifier>
Result:
―――――――
       fraud_flag                                              _prov
0               0                                  [281474976710656]
1               0                                  [281474976710657]
2               0                                  [281474976710658]
3               0                                  [281474976710659]
4               0                                  [281474976710660]
...           ...                                                ...
61236           0                 [281474976820027, 281474976820028]
61237           0                                  [281474976820029]
61238           0  [281474976820030, 281474976820031, 28147497682...
61239           0                 [281474976820033, 281474976820034]
61240           0                                  [281474976820035]

[61241 rows x 2 columns]

In [27]:
print(predictions.skb.describe_param_grid())

- learning_rate: choose_float(0.01, 0.9, log=True, name='learning_rate')
  encoder: 'MinHash'
  n_components: choose_int(5, 15, name='n_components')
- learning_rate: choose_float(0.01, 0.9, log=True, name='learning_rate')
  encoder: 'LSA'
  n_components: choose_int(5, 15, name='n_components')



In [28]:

search = predictions.skb.make_randomized_search(
    scoring="roc_auc", n_iter=8, n_jobs=4, random_state=0, fitted=True
)
search.results_

[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start
[PROVENANCE Var]: Start


AttributeError: 'Sentinels' object has no attribute 'copy'

In [ ]:
search.plot_results()

In [ ]:
import pandas as pd

new_baskets = pd.DataFrame([dict(ID="abc")])
new_products = pd.DataFrame(
    [
        dict(
            basket_ID="abc",
            item="COMPUTER",
            cash_price=200,
            make="APPLE",
            model="XXX-X",
            goods_code="239246782",
            Nbr_of_prod_purchas=1,
        )
    ]
)
search.best_learner_.predict_proba({"baskets": new_baskets, "products": new_products})